In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

import TropiGAT_functions 
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_ensemble = f"{path_work}/ficheros_28032023/ensemble_2812"
#path_ensemble = f"{path_work}/ficheros_28032023/ensemble_tailored_0612"


> Make model : 

In [2]:
dico_models, errors = TropiGAT_functions.make_ensemble_TropiGAT(path_ensemble)

In [4]:
len(dico_models)

67

> Ferriol

In [1]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.1512.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)
dpo_embeddings.index = [x if x.count("__cds") > 0 else x.replace("_cds", "__cds") for x in dpo_embeddings.index]
dpo_embeddings.index = [x.replace("__CDS","__cds") if x.count("__CDS") > 0 else x for x in dpo_embeddings.index]
dpo_embeddings.index = ['__'.join(x.split("__")[0:-1])+'_Dpo_domain' if x.count("_Dpo_domain") == 0 else x for x in dpo_embeddings.index]

> Bea

In [4]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

bea_embeddings = pd.read_csv(f"{path_project}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings.set_index([0], inplace = True)


> Towndsend

In [5]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

towndsend_embeddings = pd.read_csv(f"{path_project}/Townsed_phages.esm2.embedding.1112.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings.set_index([0], inplace = True)


>Others old

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_phages.esm2.embedding.csv", sep = "," , header = None)
others_embeddings = others_embeddings.drop([1281] , axis = 1)
others_embeddings.set_index([0], inplace = True)

namesother_df = pd.read_csv(f"{path_project}/Others/index_others.tsv", sep = "\t" , names = ["index_phage", "index_prot","prot_name"])
new_index = []
for _,index in enumerate(others_embeddings.index) : 
    i_phage = int(index.split("__")[0])
    i_prot = int(index.split("__")[1])
    prot = namesother_df[(namesother_df["index_phage"] == i_phage) & (namesother_df["index_prot"] == i_prot)]["prot_name"].values[0]
    new_index.append(prot)

others_embeddings.index = new_index
others_embeddings.index.name = 0

> Others 

In [6]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_all.esm2.embedding.csv", sep = "," , header = None)
others_embeddings.set_index([0], inplace = True)

***
> run the predictions 

In [7]:
# Run the predictions ferriol :
ferriol_predictions = {}
for dpo in dpo_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([dpo_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    ferriol_predictions[dpo] = pred

In [38]:
ferriol_predictions["K17alfa62__cds_66_Dpo_domain"]

{'KL43': 0.9988,
 'KL62': 1.0,
 'KL74': 0.552,
 'KL22': 0.6581,
 'KL128': 0.5624,
 'KL52': 0.9333,
 'KL3': 0.589,
 'KL127': 0.5839,
 'KL118': 0.8702,
 'KL46': 0.5093,
 'KL169': 0.8672,
 'KL29': 0.9861}

In [10]:
# format the results : 
ferriol_pred_formated = TropiGAT_functions.format_predictions(ferriol_predictions , sep = "__")
TropiGAT_functions.clean_print(ferriol_pred_formated)

{'K10PH82C1': {'KL10': 0.5216,
               'KL102': 0.9109,
               'KL111': 0.8457,
               'KL12': 0.915,
               'KL123': 0.9749,
               'KL127': 0.7681,
               'KL128': 0.9999,
               'KL14': 0.6084,
               'KL15': 0.8781,
               'KL151': 0.8537,
               'KL17': 0.8631,
               'KL18': 0.8807,
               'KL19': 0.6839,
               'KL2': 0.6033,
               'KL21': 0.694,
               'KL22': 0.519,
               'KL23': 0.6648,
               'KL24': 0.9981,
               'KL27': 0.8092,
               'KL29': 0.9936,
               'KL30': 0.8152,
               'KL34': 0.7711,
               'KL36': 0.87,
               'KL45': 0.829,
               'KL46': 0.9356,
               'KL48': 0.5423,
               'KL52': 0.6514,
               'KL53': 0.6696,
               'KL62': 0.7966,
               'KL63': 0.5932,
               'KL70': 0.6416,
               'KL74': 0.9494,
         

***

In [11]:
# Run the predictions Bea :
bea_predictions = {}
for dpo in bea_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([bea_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    bea_predictions[dpo] = pred

In [12]:
# format the results : 
bea_pred_formated = TropiGAT_functions.format_predictions(bea_predictions , sep = "_")
TropiGAT_functions.clean_print(bea_pred_formated)

{'A1a': {'KL10': 0.6624,
         'KL102': 0.5907,
         'KL110': 0.6816,
         'KL111': 0.9826,
         'KL12': 0.6955,
         'KL123': 0.9822,
         'KL128': 0.987,
         'KL14': 0.9181,
         'KL15': 0.5802,
         'KL21': 0.588,
         'KL23': 0.7725,
         'KL24': 0.9465,
         'KL27': 0.5112,
         'KL29': 0.9794,
         'KL30': 0.6479,
         'KL45': 0.9726,
         'KL46': 0.6381,
         'KL48': 0.5004,
         'KL70': 0.9579,
         'KL9': 0.5348},
 'A1b': {'KL102': 0.5309,
         'KL111': 0.9828,
         'KL12': 0.8481,
         'KL122': 0.5746,
         'KL123': 0.9979,
         'KL127': 0.7211,
         'KL128': 0.9908,
         'KL14': 0.9345,
         'KL149': 0.951,
         'KL17': 0.5528,
         'KL21': 0.7632,
         'KL22': 0.5946,
         'KL23': 0.6488,
         'KL24': 0.7645,
         'KL27': 0.527,
         'KL28': 0.8518,
         'KL3': 0.5774,
         'KL30': 0.7564,
         'KL36': 0.6356,
         'KL38': 0

In [13]:
# Run the predictions Towndsend :
towndsend_predictions = {}
for dpo in towndsend_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([towndsend_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    towndsend_predictions[dpo] = pred

In [ ]:
# format the results : 
towndsend_pred_formated = TropiGAT_functions.format_predictions(towndsend_predictions , sep = "_")
TropiGAT_functions.clean_print(towndsend_pred_formated)

***

In [ ]:
others_embeddings.info()

In [ ]:
others_embeddings.index

In [14]:
# Run the predictions Others :
other_predictions = {}
for index, dpo in enumerate(others_embeddings.index) :
    if dpo not in ["MN781108.1_prot_QGZ15323.1_262"] :
        try : 
            graph_dpo = TropiGAT_functions.make_query_graph([others_embeddings.loc[dpo].values])
            pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
            other_predictions[dpo] = pred
        except Exception as e :
            print(e, dpo)

 ON146449.1_prot_UPW35150.1_13
 ON146449.1_prot_UPW35138.1_1
 ON146449.1_prot_UPW35150.1_13
 ON146449.1_prot_UPW35138.1_1


In [ ]:
# format the results : 
others_pred_formated = TropiGAT_functions.format_predictions(other_predictions , sep = "_prot_")
TropiGAT_functions.clean_print(other_predictions)

***
# Write the results : 

> Others : 

In [15]:
predictions = [other_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.Others.all.results.classic_1001.tsv", "w") as outfile:
    for prediction in predictions:
        prediction_sorted = dict(sorted(prediction.items()))
        for prot in prediction_sorted:
            if prediction_sorted[prot] == "No hits" or len(prediction_sorted[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction_sorted[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

> Predictions : 

In [16]:
predictions = [ferriol_predictions , bea_predictions , towndsend_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.results.classic_1001.tsv", "w") as outfile:
    for prediction in predictions:
        for prot in prediction:
            if prediction[prot] == "No hits" or len(prediction[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

> Parse TropiGAT and Seqbased results :

In [1]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean"

tropigat_results = pd.read_csv(f"{path_project}/TropiGAT.results.classic_1001.tsv", sep = "\t", names = ["protein", "predictions_tropigat"])

# Seqbased_model.results.bit50.0101.tsv
# Seqbased_model.0101.results.tsv
# Seqbased_model.1001.results.tsv
seqbased_results = pd.read_csv(f"{path_project}/Seqbased_model.1001.results.tsv", sep = "\t", names = ["protein", "predictions_seqbased"])
seqbased_results

,protein,predictions_seqbased
0,MZ322895.1_prot_QWY13631.1_35,No hits
1,ON146449.1_prot_UPW35138.1_1,KL13: 0.659;KL2: 0.782
2,ON146449.1_prot_UPW35150.1_13,KL102: 0.691
3,MZ571831.1_prot_UEW68236.1_80,No hits
4,MZ612130.1_prot_QYC51043.1_10,No hits
...,...,...
288,NBNDMPCG_00161,KL35: 0.514
289,NBNDMPCG_00163,KL13: 0.571;KL2: 0.854
290,NBNDMPCG_00165,KL64: 0.743
291,NBNDMPCG_00166,KL14: 0.736


In [2]:
tropigat_results["protein_id"] = tropigat_results["protein"].apply(lambda x : x.split("_Dpo")[0])
seqbased_results["protein_id"] = seqbased_results["protein"].apply(lambda x : x.split("_A")[0] if x.count("_A")>0 else "__".join(x.split(",")[0].split(" ")) if x.count(",")>0 else x)

merged_df = tropigat_results.merge(seqbased_results, on='protein_id', how='inner')
merged_df["phage"] = merged_df["protein_id"].apply(lambda x : x.split("__")[0] if x.count("__")>0 else x.split("_")[0])

merged_df_sorted = merged_df.sort_values(by='phage', ascending=True)
merged_df_sorted

,protein_x,predictions_tropigat,protein_id,protein_y,predictions_seqbased,phage
173,A1a_00002,KL111:0.983 ; KL123:0.982 ; KL45:0.973 ; KL24:...,A1a_00002,A1a_00002,KL102: 0.691,A1a
158,A1a_00014,KL128:0.987 ; KL29:0.979 ; KL70:0.958 ; KL24:0...,A1a_00014,A1a_00014,KL151: 0.698,A1a
159,A1b_00048,KL46:0.994 ; KL128:0.991 ; KL149:0.951 ; KL74:...,A1b_00048,A1b_00048,KL157: 0.729,A1b
151,A1b_00036,KL123:0.998 ; KL111:0.983 ; KL128:0.982 ; KL45...,A1b_00036,A1b_00036,KL102: 0.691,A1b
182,A1c_00046,KL123:0.994 ; KL24:0.982 ; KL45:0.953 ; KL111:...,A1c_00046,A1c_00046,KL102: 0.691,A1c
...,...,...,...,...,...,...
181,S13a_00036,KL60:0.97 ; KL46:0.96 ; KL27:0.95 ; KL23:0.922...,S13a_00036,S13a_00036,KL38: 0.822,S13a
143,S13b_00058,KL47:0.994 ; KL81:0.932 ; KL74:0.917 ; KL28:0....,S13b_00058,S13b_00058,KL63: 0.867,S13b
175,S13c_00055,KL12:0.992 ; KL38:0.98 ; KL128:0.974 ; KL27:0....,S13c_00055,S13c_00055,No hits,S13c
174,S13d_00057,KL14:0.986 ; KL128:0.968 ; KL46:0.965 ; KL15:0...,S13d_00057,S13d_00057,KL14: 0.736,S13d


In [46]:
final_df = merged_df_sorted[["phage","protein_id", "predictions_seqbased", "predictions_tropigat"]]



In [47]:
final_df.to_csv(f"{path_project}/PPT_results.classic_1001.bit75.tsv", sep = "\t", header = True, index = False)

In [35]:
final_df

,phage,protein_id,predictions_seqbased,predictions_tropigat
157,A1a,A1a_00014,KL151:0.698,KL128:0.987 ; KL29:0.979 ; KL70:0.958 ; KL24:0...
172,A1a,A1a_00002,KL102:0.691,KL111:0.983 ; KL123:0.982 ; KL45:0.973 ; KL24:...
158,A1b,A1b_00048,KL157:0.729,KL46:0.994 ; KL128:0.991 ; KL149:0.951 ; KL74:...
150,A1b,A1b_00036,KL102:0.691,KL123:0.998 ; KL111:0.983 ; KL128:0.982 ; KL45...
181,A1c,A1c_00046,KL102:0.691,KL123:0.994 ; KL24:0.982 ; KL45:0.953 ; KL111:...
...,...,...,...,...
180,S13a,S13a_00036,KL38:0.822,KL60:0.97 ; KL46:0.96 ; KL27:0.95 ; KL23:0.922...
142,S13b,S13b_00058,KL63:0.867,KL47:0.994 ; KL81:0.932 ; KL74:0.917 ; KL28:0....
174,S13c,S13c_00055,No hits,KL12:0.992 ; KL38:0.98 ; KL128:0.974 ; KL27:0....
173,S13d,S13d_00057,KL14:0.736,KL14:0.986 ; KL128:0.968 ; KL46:0.965 ; KL15:0...
